In [0]:
import requests
import os
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType
from pyspark.sql.functions import to_timestamp

# Se estiver usando secrets do Databricks, substitua por dbutils.secrets.get
token = dbutils.secrets.get(scope="meu-escopo", key="token_brapi")

url = f"https://brapi.dev/api/quote/UNIP6?token={token}"
#header = {"Authorization": f"Bearer {token}"}

response = requests.get(
    url,
    #headers=header,
    params={"range": "1d", "interval": "1d"}
)

if response.status_code == 200:
    dados_brapi = response.json()["results"][0]

    # Define o schema
    schema = StructType([
        StructField("Simbolo", StringType(), True),
        StructField("Nome", StringType(), True),
        StructField("Moeda", StringType(), True),
        StructField("Fechamento_Anterior", DoubleType(), True),
        StructField("Cotacao", DoubleType(), True),
        StructField("Intervalo_52_Semanas", StringType(), True),
        StructField("Data", StringType(), True)
    ])

    # Cria o DataFrame Spark
    data = [(dados_brapi["symbol"],
             dados_brapi["shortName"],
             dados_brapi["currency"],
             dados_brapi["regularMarketPreviousClose"],
             dados_brapi["regularMarketPrice"],
             dados_brapi["fiftyTwoWeekRange"],
             dados_brapi["regularMarketTime"])]

    df_spark = spark.createDataFrame(data, schema)
    (df_spark.write
             .format("delta")
             .mode("append")
             .saveAsTable("raw.db.cotacoes")
             )
    
    print(f"Coleta realizada com sucesso, status code: {response.status_code}")
    display(df_spark)
   
else:
    print(f"Erro: {response.status_code}")
   

In [0]:
dbutils.secrets.listScopes()

In [0]:
dbutils.secrets.list("meu-escopo")
